# Implementation - Graphical model

In [37]:
import numpy  as np
import igraph as ig
import pyvis  as pv
%run ./2\ Implementation\ -\ Factors\ and\ operations.ipynb

# 1 Factor graph

### 1.1 Factor-node data structure

In [38]:
class factor_node:
    def __init__(self, name, f=None):
        self.name = name
        if type(f) is factor:
            self.__set_data(f)
        elif f is not None:
            raise Exception('Data is not factor or None')
    
    def __set_data(self, data):
        self.f = f
        
    def change_data(self, variables, distribution):
        if type(f) is factor:
            self.__set_data(factor(variables, distribution))
        else:
            raise Exception('Data is not factor')

### 1.2 Factor-variable data structure

### 1.3 Factor graph data structure

In [41]:
class factor_graph:
    def __init__(self):
        self.graph = ig.Graph()
    
    # ----------------------- Factor node actions -----------
    #def add_factor():
    #def change_factor():
    #def remove_factor():
    
    # ----------------------- Variable node actions ---------
    #def add_variable():
    #def remove_variable():

### 1.4 Factor graph from string

In [40]:
#def string2factor_graph():

### 1.5 Factor graph visualization